In [ ]:
# First version, with minimal handling of user input

# import libraries
import csv

# Define global variables (convention is that global variables are upper case)
FIELDNAMES = ['Item', 'Income(I) / Expense(E)', 'Amount', 'Date (MM-DD-YYYY)']
CSV_FILE = 'lemonade_stand_accounts.csv'

# Functions

def add_entry(csv_file):
    """Takes a new item (either income or expense) from user and writes it as a new entry to
    the csv_file.
    
    :param name: csv_file
    :type name: csv
    :return: None
    """
    item = input('Name of item: ')
    item_type = input('Type of item (Income (i) or Expense (e)): ')
    item_amount = input('Amount (enter number only): ')
    item_date = input('Date of transaction (DD-MM-YYYY): ')
    
    entry = {'Item': item.capitalize(),
             'Income(I) / Expense(E)': item_type.upper(),
             'Amount': float(abs(item_amount)),
             'Date (MM-DD-YYYY)': item_date
            }
    
    with open(csv_file,'a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=FIELDNAMES)
        writer.writerow(entry)

def display_balance(csv_file):
    """Calculates the balance across all transactions in the csv.
    
    Sum of income minus sum of expenses, with any costs that are not labeled excluded
    
    :param name: csv_file
    :type name: csv
    :return: string
    """
    with open(csv_file, newline='') as file:
        reader = csv.DictReader(file)
        income = 0
        expenses = 0
        unaccounted_for = 0
        for row in reader:
            if row['Income(I) / Expense(E)'] == 'I':
                income += float(row['Amount'])
            elif row['Income(I) / Expense(E)'] == 'E':
                expenses += float(row['Amount'])
            else:
                unaccounted_for += float(row['Amount'])
    return f'£{income - expenses} is your balance, with {unaccounted_for} unaccounted for'


def view_entries(csv_file):
    """Shows all details of all previous entries.
        
    :param name: csv_file
    :type name: csv
    :return: None
    """
    with open(csv_file, newline='') as file:
        reader = csv.DictReader(file)
        print('\nItem,', 'Inc / Exp,', 'Amount,', 'Date')
        print('=============================')
        for row in reader:
            print(f"{row['Item']}, {row['Income(I) / Expense(E)']}, {row['Amount']}, {row['Date (MM-DD-YYYY)']}")



# PROGRAM RUNS FROM HERE


try:
    lemon = open(CSV_FILE)
    lemon.close()
except FileNotFoundError:
    with open(CSV_FILE, 'w', newline='') as lemon_csv:
        writer = csv.DictWriter(lemon_csv, fieldnames=FIELDNAMES)
        writer.writeheader()

start_choice = ''
while(start_choice != "q"):
    start_choice = input("\nWhat would you like to do?\n[a] ADD a new entry to the budget tracker\n[d] DISPLAY the total balance\n[v] VIEW all previous entries\n[q] QUIT\n\nPlease select a, d, v or q :")

    if start_choice == 'a':
        add_entry(CSV_FILE)

    if start_choice == 'v':
        print(view_entries(CSV_FILE))

    if start_choice == 'd':
        print(display_balance(CSV_FILE))

print('Thank you and goodbye')

In [ ]:
# Second version, with tighter handling of user input included

# Program

# import libraries
import csv
from datetime import datetime

# Define global variables (convention is that global variables are upper case)
FIELDNAMES = ['Account Name', 'Item', 'Income(I) / Expense(E)', 'Amount', 'Date (MM-DD-YYYY)']
CSVFILE = 'lemonade_stand_accounts.csv'
DATEFORMAT = "%d-%m-%Y"

# Functions

# Exception handling functions

def handle_empty_field(field, question):
    while not field:
        field = input(question)
    return field

def handle_item_type(item_type):
    valid_types = {'I', 'E'}
    while item_type.upper() not in valid_types:
        item_type = input('Type of item (Income (i) or Expense (e)): ')
    return item_type

def handle_item_amount(item_amount):
    while True:
        try:
            item_amount = float(abs(item_amount))
        except ValueError:
            print("Please re-input the amount, and only use numbers.")
            item_amount = input("Amount (enter number only): ")
        else:
            break
    return item_amount

def handle_item_date(item_date):
    while True:
        try:
            bool(datetime.strptime(item_date, DATEFORMAT))
        except ValueError:
            item_date = input('Date of transaction (DD-MM-YYYY): ')
        else:
            break
    return item_date


def add_entry(csv_file):
    """Takes a new item (either income or expense) from user and writes it as a new entry to
    the csv_file.
    
    :param name: csv_file
    :type name: csv
    :return: None
    """
    account_name = handle_empty_field(input('What is the account name? '), 'What is the account name? ')
    item = handle_empty_field(input('Name of item: '), 'Name of item: ')
    item_type = handle_item_type(input('Type of item (Income (i) or Expense (e)): '))
    item_amount = handle_item_amount(input('Amount (enter number only): '))
    item_date = handle_item_date(input('Date of transaction (DD-MM-YYYY): '))
    
    entry = {'Account Name': account_name,
             'Item': item.capitalize(),
             'Income(I) / Expense(E)': item_type.upper(),
             'Amount': item_amount,
             'Date (MM-DD-YYYY)': item_date
            }
    
    with open(csv_file,'a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=FIELDNAMES)
        writer.writerow(entry)

def display_balance(csv_file):
    """Calculates the balance across all transactions in the csv.  Can provide data for a
    particular username, or all transactions.
    
    Sum of income minus sum of expenses, with any costs that are not labeled excluded.
    
    :param name: csv_file
    :type name: csv
    :return: string
    """
    account = input('Which account do you want to view? If all, just leave blank. ')
    year = input('Which year do you want to view? If all, just leave blank. ')
    month = input('Which month do you want to view? if all, just leave blank. ')
    
    with open(csv_file, newline='') as file:
        reader = csv.DictReader(file)
        income = 0
        expenses = 0
        unaccounted_for = 0
        
        def filter_rows(row):
            """Filter containing conditions for filtering rows in the csv file that have
            been read in."""
            
            if account and period:
                return row['Account Name'] == account and datetime.strptime(str(row['Date (MM-DD-YYYY)']), DATEFORMAT).year == int(year)
            elif account:
                return row['Account Name'] == account
            elif period:
                return datetime.strptime(str(row['Date (DD-MM-YYYY)']), DATEFORMAT).year == int(period)
            else:
                return row['Account Name'] #returns all rows if both input fields are left blank

        for row in filter(filter_rows, reader):
            if row['Income(I) / Expense(E)'] == 'I':
                income += float(row['Amount'])
            elif row['Income(I) / Expense(E)'] == 'E':
                expenses += float(row['Amount'])
            else:
                unaccounted_for += float(row['Amount'])
    print(f'£{income - expenses} is the balance, with {unaccounted_for} unaccounted for')


def view_entries(csv_file):
    """Shows all details of all previous entries.
        
    :param name: csv_file
    :type name: csv
    :return: None
    """
    with open(csv_file, newline='') as file:
        reader = csv.DictReader(file)
        print('\nItem,', 'Inc / Exp,', 'Amount,', 'Date')
        print('=============================')
        for row in reader:
            print(f"{row['Account Name']}, {row['Item']}, {row['Income(I) / Expense(E)']}, {row['Amount']}, {row['Date (DD-MM-YYYY)']}")

# PROGRAM RUNS FROM HERE


try:
    lemon = open(CSVFILE)
    lemon.close()
except FileNotFoundError:
    with open(CSVFILE, 'w', newline='') as lemon_csv:
        writer = csv.DictWriter(lemon_csv, fieldnames=FIELDNAMES)
        writer.writeheader()

start_choice = ''
while(start_choice != "q"):
    start_choice = input("\nWhat would you like to do?\n[a] ADD a new entry to the budget tracker\n[d] DISPLAY the total balance\n[v] VIEW all previous entries\n[q] QUIT\n\nPlease select a, d, v or q :")

    if start_choice == 'a':
        add_entry(CSVFILE)
        
    elif start_choice == 'd':
        display_balance(CSVFILE)

    elif start_choice == 'v':
        view_entries(CSVFILE)
    
    elif start_choice == 'p':
        p_and_l(CSVFILE)

print('Thank you and goodbye')

**End**